# DM - Système de décision

## PARTIE 1 : Gurobi

In [49]:
import numpy as np
from gurobipy import Model
from gurobipy.gurobipy import GRB
import random

On génère un dataset qui pourra être résolu par notre solveur. 

Il nous faut donc des valeurs cohérentes de notes par rapport aux coefficients des différentes matières ainsi que des frontières entre les différentes classes (Accepté, Refusé, ...).

In [50]:
def check_class(student, w, frontiers_list, lambdou):
    """
    Retourne la classe d'un étudiant
    :param student: {np.array} Notes d'un élève donné
    :param w: {np.array} Coefficients des matières
    :param frontiers_list: {np.array[]} Frontières des notes pour chaque matière
    :param lambdou: {int} Seuil de majorité
    :return: {int} Retourne la classe d'un étudiant (ex: 0 pour refusé, 1 pour accepté)
    """
    #On part de la classe 0 et on incrémente à chaque fois que la somme des coeffs des matières que l'élève valide pour un niveau donné dépasse lambda
    student_class = 0

    #frontiers_list contient une liste de notes frontières pour chaque matières
    #on transforme pour avoir une liste de frontières des matières pour chaque classe
    frontiers_t = frontiers_list.T

    for frontiers in frontiers_t:
        coeffs_sum = ((student >= frontiers) * w).sum()
        if coeffs_sum >= lambdou:
            student_class += 1
        else:
            break

    return student_class

In [51]:
def convert_dataset(S, y, classes_count):
    """
    :param S: {np.array[]} Notes des élèves
    :param y: {list} Classe des élèves
    :param classes_count: {int} Nombre de classes dans le dataset
    :return: {list(np.array()[]} Liste de classes_count matrices de notes
    """
    X = [[] for i in range(classes_count)]

    for i in range(len(y)):
        student_class = y[i]
        X[student_class].append(list(S[i]))

    return [np.array(matrix) for matrix in X]

In [52]:
def generate_dataset(students_count, subjects_count, classes_count):
    """
    Crée un dataset de notes d'étudiant et de leur classe attribuée
    :param students_count: {int} Nombre d'étudiants
    :param subjects_count: {int} Nombre de matières
    :param classes_count:  {int} Nombre de classes (ex: 2 pour {Accepté, Refusé})
    :return: {list(np.array()[]} Liste de classes_count matrices de notes
    """
    #On génère les coefficients des différentes matières, dont la somme vaut 1
    w_ects = np.random.randint(0, 30, subjects_count)
    w = w_ects / w_ects.sum()
    print("\nLes coeffs sont = ", w)

    #Les frontières entre chaque classe (si 2 classes A et R, seulement 1 frontière)
    frontiers_list = np.zeros((subjects_count, classes_count - 1))
    for i in range(subjects_count):
        frontiers = np.sort(random.sample(list(range(1, 21)), classes_count - 1))
        frontiers_list[i,] = frontiers
    print("\nLes frontières sont = ", frontiers_list)

    #La somme des coeffs minimales à avoir pour être dans une classe donnée
    lambdou = random.uniform(0.5, 1)
    print("\nlambda = ", lambdou)

    #On génère les notes des élèves (1 ligne par élève)
    S = np.random.randint(0, 21, (students_count, subjects_count))
    print("\nLes notes des élèves :")
    print(S)

    #On regarde à quelle classe est associé chaque élève en fonction de ses notes
    y = []
    for student in S:
        student_class = check_class(student, w, frontiers_list, lambdou)
        y.append(student_class)

    print("\nLes classes des élèves :")
    print(y)

    #On veut retourner une matrice par classe 
    #Si 2 classes A et R : A contient les notes des élèves qui sont admis et R les notes des élèves refusés
    X = convert_dataset(S, y, classes_count)

    return X


In [166]:
X = generate_dataset(10, 4, 3)


Les coeffs sont =  [0.02702703 0.35135135 0.16216216 0.45945946]

Les frontières sont =  [[ 6. 20.]
 [ 4. 12.]
 [15. 16.]
 [14. 20.]]

lambda =  0.9376180086891268

Les notes des élèves :
[[ 8  4 19 18]
 [20 14 16  3]
 [ 9 16  3  4]
 [19  8 17 15]
 [12 18 12  4]
 [15 13 16  7]
 [15  0  1  7]
 [11  5 19  1]
 [14  6  0  7]
 [ 7 12  8 19]]

Les classes des élèves :
[1, 0, 0, 1, 0, 0, 0, 0, 0, 0]


In [160]:
def solve(X, students_count, subjects_count, classes_count, verbose=0):
    """
    :param X: {list(np.array()[]} Liste de classes_count matrices de notes
    :param students_count: {int} Nombre d'étudiants
    :param subjects_count: {int} Nombre de matières
    :param classes_count:  {int} Nombre de classes (ex: 2 pour {Accepté, Refusé})
    :param verbose:
    :return:
    """
    R, *L, A = X

    # Instanciation du modèle
    m = Model("PL modeling")

    #------------------VARIABLES------------------

    # Vecteur des coefficients des différentes matières
    w = m.addMVar(shape=subjects_count, lb=0, ub=1, name="w")

    c_accepted = m.addMVar(shape=(len(A), subjects_count), lb=0, ub=1, name="c_accepted")
    c_refused = m.addMVar(shape=(len(R), subjects_count), lb=0, ub=1, name="c_refused")

    delta_accepted = m.addMVar(shape=(students_count, subjects_count), vtype=GRB.INTEGER, lb=0, ub=1,
                               name="delta_accepted")
    delta_refused = m.addMVar(shape=(students_count, subjects_count), vtype=GRB.INTEGER, lb=0, ub=1, name="delta_refused")

    sigma_accepted = m.addMVar(shape=len(A), name="x", vtype=GRB.CONTINUOUS)
    sigma_refused = m.addMVar(shape=len(R), name="y", vtype=GRB.CONTINUOUS)

    lambd = m.addVar(name="lambda", lb=0.5, ub=1)

    b = m.addMVar(shape=subjects_count, name="b", lb=0, ub=20)

    alpha = m.addVar(name="alpha")

    m.update()

    epsilon = 0.01

    M = 150
    
    
    #------------------CONTRAINTES------------------

    # Validation constraint with sigma
    for index, student_point in enumerate(c_accepted):
        m.addConstr(sum(student_point) - sigma_accepted[index] + epsilon == lambd)

    # Refusal constraint with sigma
    for index, student_point in enumerate(c_refused):
        m.addConstr(sum(student_point) == lambd - sigma_refused[index])

    # Alpha is inferior than all sigmas
    for offset in (*sigma_accepted, *sigma_refused):
        m.addConstr(alpha <= offset)

    # # cij is inferior than wi
    # for matrix in (c_accepted ,c_refused):
    #     for student in matrix:
    #         for index, point in enumerate(student):
    #             m.addConstr(point <= w[index])

    accepted_tuple = (c_accepted, delta_accepted, A)
    refused_tuple = (c_refused, delta_refused, R)

    for c, delta, grades in (accepted_tuple, refused_tuple):
        for i in range(c.shape[0]):
            for j in range(c.shape[1]):
                m.addConstr(c[i, j] <= w[j])
                m.addConstr(c[i, j] <= delta[i, j])
                m.addConstr(c[i, j] >= delta[i, j] - 1 + w[j])
                m.addConstr(M * delta[i, j] + epsilon >= grades[i, j] - b[j])
                m.addConstr(M * (delta[i, j] - 1) <= grades[i, j] - b[j])

    m.addConstr(sum(w) == 1)

    m.update()

    if verbose:
        m.params.outputflag = 0

    #Fonction objectif
    m.setObjective(alpha, GRB.MAXIMIZE)
    m.update()

    m.optimize()
    print("====== Model solved =====")
    print(f"alpha = {alpha.X}")
    print(f"lambda = {lambd.X}")
    print(f"b = {b.X}")

    print("\nLes coefficients de notre solution optimale sont :")
    print(f"__________________")

    print(f"| Matière  |coeff|")

    for index, coef in enumerate(w.X):
        print(f"|matière {index + 1} | {coef.round(2)} |")
    print(f"__________________")

    return A, R, w.X, lambd.X, b.X


### Exemple 1 : 

20 élèves, 5 matières et 2 classes (Accepté ou Refusé) 

In [161]:
X = generate_dataset(20, 5, 2)


Les coeffs sont =  [0.30864198 0.17283951 0.16049383 0.16049383 0.19753086]

Les frontières sont =  [[ 9.]
 [13.]
 [ 4.]
 [13.]
 [14.]]

lambda =  0.6510852823915507

Les notes des élèves :
[[18 13  1  4 15]
 [ 5  8  7 17 15]
 [14 15  4 16  0]
 [ 7 18 15 20  7]
 [ 8  8 13  1  1]
 [ 5 10 12  9  4]
 [ 1  6  5  5 12]
 [12 20 12 10 14]
 [ 8 12 16  4 11]
 [ 2 15 17 16  5]
 [ 7  3 20 10 16]
 [12  5  3 18 16]
 [ 1 18  2  4  0]
 [15  4  8 11  7]
 [ 4  3 18  6 10]
 [15  6  8  3  2]
 [ 8 11  9 15  9]
 [19 20  7  9 11]
 [ 0 11  5 14 15]
 [ 2  5 15 19  6]]

Les classes des élèves :
[1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]


In [162]:
accepted, refused, weigths, lambd, b = solve(X, 20, 5, 2, verbose=0)

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 541 rows, 332 columns and 1285 nonzeros
Model fingerprint: 0xe3d53617
Variable types: 132 continuous, 200 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e-01, 2e+01]
  RHS range        [1e-02, 2e+02]
Presolve removed 34 rows and 120 columns
Presolve time: 0.00s
Presolved: 507 rows, 212 columns, 1217 nonzeros
Variable types: 107 continuous, 105 integer (100 binary)
Found heuristic solution: objective 0.0040000
Found heuristic solution: objective 0.0050000

Root relaxation: objective 5.050000e-01, 184 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.50500    0   19    0.00500    0.50500      -

In [163]:
print((accepted >= b) @ weigths.T >= lambd)

[ True  True  True  True]


In [164]:
print((refused >= b) @ weigths.T >= lambd)

[False False False False False False False False False False False False
 False False False False]


**Conclusion :**

Avec 2 classes (Accepté, Refusé), notre MRSort fonctionne et nous renvoie une solution faisable pour notre dataset.